In [1]:
# Importing all the modules 
import torch 
import torch.nn as nn 
import torch.optim as optim

import torchvision 
from torch.utils.data import DataLoader,Dataset,TensorDataset
import torch.utils.data as utils
from torchvision import transforms 
import pandas as pd




In [2]:
import matplotlib.image as mpimg
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset

In [3]:
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

In [4]:
# Defining the dir
data_dir = "aerial-cactus-identification/"
train_dir = data_dir + "train/"
test_dir = data_dir + "test"

In [5]:
# Loading the CSV
labels = pd.read_csv("aerial-cactus-identification/train.csv")
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [6]:
# K- Fold

df = labels.sample(frac=1).reset_index(drop = True)                      # Reset Index
df['kfold'] = -1                                                         # Intitialize new col
y = labels.has_cactus.values                                             
kf = model_selection.StratifiedKFold(n_splits = 5,shuffle = True)        # Intitialize Fold no.
idx = kf.get_n_splits(X=df,y=y)                                          # Splitting the dataset
print(idx)
for fold,(x,y) in enumerate(kf.split(X=df,y=y)):
    df.loc[y,'kfold'] = fold


5


In [7]:
import os 

class ImageData(Dataset):
    def __init__(self,df,df_target,data_dir,transform):
        super().__init__()
        
        self.df = df
        self.transform = transform
        self.data_dir = data_dir
        self.df_target = df_target
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        print(len(self.df),index)
        img_name = self.df[index]
        label = self.df_target[index]
        
        img_path = os.path.join(self.data_dir,img_name)
        image = mpimg.imread(img_path)
        image = self.transform(image)
        return image,label
        
    

In [8]:
data_trans = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.ToTensor()
])

#train_data = ImageData(df = labels,data_dir=train_dir,transform=data_trans)



In [10]:
#!pip install efficientnet_pytorch


In [11]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')


In [12]:
for param in model.parameters():
    param.requires_grad = True

In [13]:
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs,1)

optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()

In [14]:
def train(fold):
    batch_t = 2
    batch_v = 2
    best_score = 0
    train_df = df[df.kfold!=fold].reset_index(drop=True)
    valid_df = df[df.kfold==fold].reset_index(drop=True)
    
    train_im = train_df.id.values.tolist()
    train_y = train_df.has_cactus.values
    valid_im = valid_df.id.values.tolist()
    valid_y = valid_df.has_cactus.values
    
    
    train_data = ImageData(df = train_im,df_target = train_y,data_dir=train_dir,transform=data_trans)
    trainloader = DataLoader(dataset = train_data,batch_size = 2)
    
    valid_data = ImageData(df = valid_im,df_target = valid_y,data_dir=train_dir,transform=data_trans)
    validloader = DataLoader(dataset = train_data,batch_size = 2)
    
    loss_log = []
    valid_loss = []
    valid_loss = np.Inf


    for epoch in range(5):
        model.train()
        batch = 0
        for ii ,(data,target) in enumerate(trainloader):
            target = target.float()

            optimizer.zero_grad()
            output = model(data)

            m = nn.Sigmoid()
            loss = loss_func(m(output),target)
            loss.backward()

            optimizer.step()
            batch+=1
            
            if batch%100==0 : 
                print("EPOCH {}  Loss {}  batch  {}".format(epoch,loss.item(),batch))
            if ii%1000 == 0:
                loss_log.append(loss.item())
        print('Epoch: {} - Loss: {:.6f} v_Loss: {:.6f}'.format(epoch + 1, loss.item(),v_loss.item))


        model.eval()
        batch = 0
        for ii,(data,target) in enumerate(validloader):
            target = target.float()
            output = model(data)
            
            batch+=1
            m = nn.Sigmoid()
            v_loss = loss_func(m(output),target)
            if ii%1000 == 0:
                valid_loss.append(v_loss.item())
                
            if batch%100==0 : 
                print("EPOCH {}  Loss {}  batch  {}".format(epoch,loss.item(),batch))


        print('Epoch: {} - Loss: {:.6f} v_Loss: {:.6f}'.format(epoch + 1, loss.item(),v_loss.item))
    

In [15]:
train(0)
train(1)
train(2)
train(3)
train(4)

14000 0
14000 1


/home/kunal/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


14000 2
14000 3
14000 4
14000 5
14000 6
14000 7
14000 8
14000 9
14000 10
14000 11
14000 12
14000 13
14000 14
14000 15
14000 16
14000 17
14000 18
14000 19
14000 20
14000 21


KeyboardInterrupt: 

In [ ]:
submit = pd.read_csv('../input/sample_submission.csv')
test_data = ImageData(df = submit, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)

In [ ]:
predict = []
model.eval()
for i, (data, _) in enumerate(test_loader):
    output = model(data)    

    pred = torch.sigmoid(output)
    predicted_vals = pred > 0.5
    predict.append(int(predicted_vals))
    
submit['has_cactus'] = predict
submit.to_csv('submission.csv', index=False)